<h1>ai知识图谱生成器<h1>

<h3>大纲<h3>

<h5>本项目旨在，将非结构化文本文档（没有固定格式或预定义数据模型）通过自己选择的LLM以“主体-谓语-客体”（SPO/主谓宾）三重奏的形式提取知识，并将关系可视化为知识图谱。<h5>  
包含以下六个流程：输入文本--分块处理--抽取三元组，提取知识--实体标准化--关系推断--可视化  

<h3><span style="color : blue">本体如何构建</span><h3>  
 以main.py文件为主流程调度文件，通过调用LLM.py、config.py、prompts.py、text_utils.py、config.py、entity_standardization.py等文件中的函数进行整个流程的运行.其中通过config.py文件，获取主流程的配置。llm.py文件帮助调用LLM，并在prompts.py文件的辅助下生成响应，并提取响应中的JSON数据。text_utils.py文件帮助对整个文本进行分块。entity_standardization.py文件对初步处理后的三元组进行标准化处理和关系推理。最后通过visualization.py文件辅助进行可视化。


<h5><span style="color : red">主流程构建及相关文件作用解析: </span> 

1.config.toml(配置文件，模型参数)  

2.generate-graph.py(命令行入口脚本，可以通过命令行直接运行)  

3.main.py (核心逻辑，主流程调度)  

4.llm.py(大语言模型交互逻辑)  

5.entity_standardization.py(实体标准化，统一标准)  

6.visualization.py(生成交互式图谱)  

7.text_utils.py(文本分块规则，如何分块)  

8.prompts.py（系统提示词构建）  

<h5>

<h5>一、config.toml(配置文件，模型参数)<h5>  
    二、main.py（核心逻辑，主流程调度）   
     核心流程主要包括：文本分块处理、调用LLM和提取三元组、清洗和增强数据、生成交互式图谱  

1.导入库、配置环境  
<span style="color : red">2. process_with_llm（如何提取三元组、定义处理逻辑）通过调用自己选择的大模型，生成结构化三元组(spo) </span>   
  （1）加载LLM系统提示词和用户提示词  
  （2）配置LLM参数：（config.toml文件，可以通过调节temperature参数的数值来控制生成文本时的随机性）  
  （3）解析JSON数据，过滤无效三元组，限制谓词（关系）长度：引入valid_triples、invalid_count、两个参数。valid_triples表示三元组有效，若符合有效条件，则直接进行合并输出，若不符合条件，则invalid_count数量加1，若invalid_count数量大于一，则会输出警告“过滤了n个无效三元组”，若没有valid_triples，则会输出“没有找到有效三元组”，同时限制谓语（关系）的长度  
<span style="color : red">3.process_text_in_chunks（分块处理文本）</span>  
  （1）进行分块：读取config中的分块参数，然后通过text_utils.py文件中的分块函数**chunk_text**函数对full_text进行分块  
  （2）初始三元组提取：遍历文本块，调用**process_with_llm**函数处理当前块提取三元组，同时为每个三元组添加块编号  
  （3）实体标准化（可选）：通过config文件中的配置判断是否进行实体标准化，若进行实体标准化，则通过引用entities_standardization文件中的**standardize_entities**函数进行实体标准化，将相同个体的不同表述统一。  
  （4）关系推理（可选）：通过config文件中的配置判断是否进行关系推理，若进行关系推理，则通过引用entity_standardization文件中的**infer-relationship**函数进行关系推理，补充文本中隐含的关系  
 <span style="color : red"> 4. 辅助函数：get_unique_entities（遍历所有三元组去重，提取唯一实体）</span>  
 <span style="color : red">5.主函数main（程序入口）：</span>可以通过命令行参数调整模式，读取输入文本或测试文本，调用process_text_in_chunks提取三元组，保存原始数据，生成可视化HTML。  
  

 三、llm.py（大语言模型交互逻辑）  
   （1）call_llm：调用大语言模型，根据user_prompt和system_prompt生成响应  
   （2）extract_json_from_text：处理模型生成的不干净的响应，提取有效的JSON数组
  


<h3><span style="color : blue">信息抽取实现思路<h3>

<h5>

1.将用户输入待处理文本分块  
2.通过process_with_LLM函数对分块文本进行处理  
 调用LLM在系统提示词的辅助下，生成响应，对生成的响应提取JSON结构数据，对提取的数据进行初步清洗  
3.对处理的后的数据进行标准化处理  
4.进行关系推理  
5.通过get_unique_entities提取唯一的实体  
6.生成可视化知识图谱
<h5>

<h3><span style="color : blue">实体标准化<h3>

<h5>  实体标准化共分为两个阶段：将文本小写，去除停用词，生成一个标准，按照该标准进行分组，对同一组内的进行筛选，在此基础上还有可能会出现以下情况，即“Apple"、"apple inc"没有被分到同一组，所以要进行第二次标准化，也就是基于包含关系和词干相似度进行标准化  

<span style="color : red">第一次实体标准化：</span>（对所有实体进行“去除停用词，转小写”处理，并以此为分组标准，若处理后的实体相同，则分为同一组，处理后的实体作为索引，原实体（变体）为值，而第一次标准化处理实际上是对同一分组中的进行处理，若同一分组中，只有一个值，则以该值作为代表性实体，若有多个值，则统计每个值在原三元组中出现的频率，以频率出现最多长度最短的作为代表性实体，并将上述所有索引与值的映射关系放入到内部工具字典standardized_entities中）  
（1）验证是否为有效三元组（同时包含subject、object、predicate三个字段以及为字典类型），若符合条件，则存入列表，若不符合，统计无效三元组数量并提示  
（2）将所有实体转换为小写并去重，放入到all_entities空集合当中    
（3）定义分组标准的规则：将文本小写，定义停用词，用正则提取单词，插入空格，去除停用词    
（4）对所有小写化后的实体按照长度从长到短进行排序，防止优先使用短实体作为标准  
（5）将实体在排序后的实体中进行遍历，同时进行第一次实体标准化（即：将文本转化为小写并去除停用词后相同的归为一组）    
（6）以标准化后的实体作为索引，将标准化前的实体作为值放在同一临时分组entity_groups当中（将实体按照标准化后的实体作为分类标准，放入到一个组中。）    
（7）对每一个临时分组中的实体选择一个最具有代表性的实体，若该临时分组中只有一个实体（值），则直接选择该实体。若有多个实体（值），则将实体在原始三元组中出现频率最多的，长度最短的作为代表性实体  
 
 <span style="color : red">第二次标准化（基于词根关系）:</span>获取第一次标准化后字典standardized_entities中的值（set去重），并按长度从小到大进行排序放入sorted_standards中，通过两层循环遍历sorted_standards中的所有实体，检查是否存在包含关系，若存在，则选取较短的作为标准（一个实体是另一个的子集），相似度高低（对于长度大于4的，取前四个词，并筛选出重叠部分占最长实体的50%以上的实体<h5>